In [1]:
import datetime
import math
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

pd.set_option('display.max_colwidth', 50)

Target_input = pd.read_csv(
    "Target_input.csv", encoding="UTF-8").set_index(['Time'])

In [2]:
print(len(Target_input))
Target_input

24


,Demand[kWh],Forecast_PV[Wh/unit]
Time,,
2019/5/7 0:00,0.288,0.000000
2019/5/7 1:00,0.288,0.000000
2019/5/7 2:00,0.288,0.000000
2019/5/7 3:00,0.288,0.000000
2019/5/7 4:00,0.288,0.000000
2019/5/7 5:00,0.288,13.312500
2019/5/7 6:00,0.288,122.742552
2019/5/7 7:00,0.374,312.615258
2019/5/7 8:00,1.331,520.452192


In [3]:
#parameters for calc cost
initial_cost_parameters = {"PV_investment[yen/kWh]": 20,
              "battery_investment[yen/kWh]": 50,
              "diesel_investment[yen/kWh]": 70,
              }

#Input parameters
number_demand=1
pv_capacity_per_unit=245
np_demand=number_demand * Target_input['Demand[kWh]'].values
np_PV_efficient=Target_input['Forecast_PV[Wh/unit]'].values/1000

In [4]:
#temporary fitness variable parameters
pv_cap_max =245  #W
battery_cap_max=1  #kWh
diesel_max=5  #kWh
#temporary parameters
h=0
trashed_power=0
battery_max =  1.0 * battery_cap_max
battery_min=0.3* battery_cap_max #kWh
p_battery=0.5*battery_max

In [5]:
def pv_generate(h,pv_cap_max):
    pv=(pv_cap_max/pv_capacity_per_unit)*np_PV_efficient[h]
    return pv

In [6]:
#pv=pv_generate(h,pv_cap_max)
#print(pv,np_demand[h])

In [7]:
def flowchart(h, p_battery):
    
    # initialize some parameters
    pv = pv_generate(h, pv_cap_max)
    check = "True"
    p_diesel = 0
    trashed_power = 0
    root= str(h)+"h"
    # 太陽光の発電量が需要より多いか確認
    if pv > np_demand[h]:
        # 太陽光の発電量が需要より多い時の処理
        # バッテリーが過充電にならないかのチェック、過充電の場合、余った電気は捨てる。
        if p_battery+(pv-np_demand[h]) <= battery_max:
            p_battery = p_battery+(pv-np_demand[h])
            root=str(h)+"h: "+"Battery charge without diesel."
        else:
            trashed_power = ((pv-np_demand[h])+p_battery) - battery_max
            p_battery = battery_max
            root=str(h)+"h: "+"Fullcharge ,no diesel, trashed "\
                 + str(round(trashed_power, 2))+"[kWh]."

    else:
        # 太陽光の発電量が需要より少ない時の処理
        # ディーゼルを使うかチェック
        if p_battery > (np_demand[h]-pv) and p_battery-(np_demand[h]-pv) > battery_min:
            p_battery = p_battery-(np_demand[h]-pv)
        else:
            p_diesel = np_demand[h]-pv-(p_battery-battery_min)
            # ディーゼルの容量を超える場合はエラー
            if p_diesel > diesel_max:
                check = 'False'
                root=str(h)+"h: "+"Error! diesel capacity is over!"
            else:
                root= str(h)+"h: "+"discharging " + str(p_battery-battery_min)\
                      +"[kWh], diesel " + str( round(p_diesel, 3) ) +"[kWh]."
                p_battery = battery_min

    parameters = {"hour": h,
                  "pv": pv,
                  "battery state": p_battery,
                  "demand": np_demand[h],
                  "diesel power": p_diesel,
                  "trashed power": trashed_power,
                  "Check":check,
                  "root":root,
                    }

    return parameters, check, root

In [8]:
def loop_flowchart(h, p_battery):
    #からのデータフレームを作成
    df = pd.DataFrame()
    for h in range(24):
        flowchart_parameters, check, root=flowchart(h, p_battery)
        p_battery=flowchart_parameters['battery state']
        df=pd.concat([df,pd.io.json.json_normalize(flowchart_parameters)])

    #ループ内でfalseがあるかチェック　falseがあるとtotal_checkにfalseが入る。
    total_check = not("False" in df['Check'].values)

    variables = {"pv_cap_max": pv_cap_max,
                 "battery_cap_max": battery_cap_max,
                 "battery_max": battery_max,
                 "battery_min": battery_min,
                 "diesel_max": diesel_max,
                 "demand_sum": df['demand'].sum(),
                 "pv_power_sum": df['pv'].sum(),
                 "diesel_power_sum ": df['diesel power'].sum(),
                 "trashed_power_sum ": df['trashed power'].sum(),
                    }
    
    return df ,total_check, variables

In [9]:
df ,total_check, variables =loop_flowchart(h, p_battery)

In [10]:
print(total_check)
print(variables)
df

True
{'pv_cap_max': 245, 'battery_cap_max': 1, 'battery_max': 1.0, 'battery_min': 0.3, 'diesel_max': 5, 'demand_sum': 19.343999999999998, 'pv_power_sum': 7.1802110181, 'diesel_power_sum ': 12.1517982083, 'trashed_power_sum ': 0.18800922639999995}


,Check,battery state,demand,diesel power,hour,pv,root,trashed power
0,True,0.300000,0.288,0.088000,0,0.000000,"0h: discharging 0.2[kWh], diesel 0.088[kWh].",0.000000
0,True,0.300000,0.288,0.288000,1,0.000000,"1h: discharging 0.0[kWh], diesel 0.288[kWh].",0.000000
0,True,0.300000,0.288,0.288000,2,0.000000,"2h: discharging 0.0[kWh], diesel 0.288[kWh].",0.000000
0,True,0.300000,0.288,0.288000,3,0.000000,"3h: discharging 0.0[kWh], diesel 0.288[kWh].",0.000000
0,True,0.300000,0.288,0.288000,4,0.000000,"4h: discharging 0.0[kWh], diesel 0.288[kWh].",0.000000
0,True,0.300000,0.288,0.274688,5,0.013312,"5h: discharging 0.0[kWh], diesel 0.275[kWh].",0.000000
0,True,0.300000,0.288,0.165257,6,0.122743,"6h: discharging 0.0[kWh], diesel 0.165[kWh].",0.000000
0,True,0.300000,0.374,0.061385,7,0.312615,"7h: discharging 0.0[kWh], diesel 0.061[kWh].",0.000000
0,True,0.300000,1.331,0.810548,8,0.520452,"8h: discharging 0.0[kWh], diesel 0.811[kWh].",0.000000
0,True,0.300000,1.274,0.581854,9,0.692146,"9h: discharging 0.0[kWh], diesel 0.582[kWh].",0.000000


In [11]:
def calc_cost(variables, initial_cost_parameters):
    #initial_cost_parameters["PV_investment[yen/kWh]"]
    #initial_cost_parameters["battery_investment[yen/kWh]"]
    #initial_cost_parameters["diesel_investment[yen/kWh]"]
    total_cost = variables["pv_cap_max"] * initial_cost_parameters["PV_investment[yen/kWh]"] \
                +variables["battery_cap_max"] * initial_cost_parameters["battery_investment[yen/kWh]"] \
                +variables["diesel_max"] * initial_cost_parameters["diesel_investment[yen/kWh]"]
    
    return total_cost

In [12]:
calc_cost(variables, initial_cost_parameters)

5300